In [1]:
import numpy as np
import os
import sys
import astropy as ast
from astropy.io import ascii
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import scipy as sp
from astropy.timeseries import LombScargle
from astropy import units as u
import gc

In [2]:
from view_and_clean import get_telescope, find_header_line, modified_zscore, df_extract, individual_plotter, grid_plotter, offset_corrector, offset_corrector_window
from metrics import mean_med_flux, fit_plotter_flux, curve_fitter_flux,\
    largest_amplitude, offset_warning, chi, percentile_amplitude, plot_percentile_amplitude, \
        compute_lomb_scargle, plot_periodogram, plot_phase_fold, amplitude_per_period, amplitude_per_period_plot

### Extracting data from files and plotting:

In [3]:
# coords = pd.read_csv('ysg_coords_with_lcs.dat', comment='#', sep="\\s+", names=['RA', 'DEC'])
# coords = pd.read_csv('/Users/shannonbowes/Research/ysg/candidates_and_lcs/final_smc_ysgcands.csv', comment='#', sep="\\s+", names=['ra', 'dec'])
coords = pd.read_csv('merged_smc_lmc_coords.csv', comment='#', sep="\\s+", names=['ra', 'dec'])


In [4]:
results = []

### New data:

In [6]:
# Create DataFrame structure once before the loop
columns = [
    'RA', 'DEC', 'overall_mean', 'overall_median', 'overall_mean_mag', 'overall_median_mag',
    'offset_flag', 'chi_squared_68', 'chi2_threshold_68', 'chi_flag_68',
    'chi_squared_95', 'chi2_threshold_95', 'chi_flag_95', 
    'chi_squared_997', 'chi2_threshold_997', 'chi_flag_997',
    'amplitude_5', 'lower_percentile_5', 'upper_percentile_5', 'mag_amplitude_5',
    'amplitude_1', 'lower_percentile_1', 'upper_percentile_1', 'mag_amplitude_1',
    'largest_amp', 'best_period', 'alarm_level_flag', 'std_amplitude'
]

# Initialize DataFrame with NaN values and pre-populate coordinates
results_df = pd.DataFrame(index=range(848), columns=columns, dtype=float)
results_df['RA'] = coords['ra'].values
results_df['DEC'] = coords['dec'].values

# Main analysis loop
for target in range(0, 848):
    print("Target: ", target)
    
    try:
        df, telescopes = offset_corrector(target, additive=False, show=False)
        # if target in [375, 376, 377, 378, 379, 380]:
        #     print(f"\n=== DETAILED TARGET {target} ANALYSIS ===")
        #     print(f"Dataset: {'SMC' if target <= 376 else 'LMC'}")
        #     print(f"DataFrame shape: {df.shape}")
        #     print(f"Telescopes: {telescopes}")
        #     print(f"Time range: {df['HJD'].min():.1f} to {df['HJD'].max():.1f}")
        #     print(f"Unique telescope count: {len(telescopes)}")
        #     print(f"Data points per telescope: {[len(df[df['telescope']==t]) for t in telescopes]}")
        
        # analysis
        overall_mean, means, overall_median, medians, overall_mags_mean, overall_mags_median, mags_means, mags_medians = mean_med_flux(target, df=df, telescopes=telescopes)
        chi_squared_95, chi2_threshold_95, dof_95, chi_flag_95 = chi(target, df=df, telescopes=telescopes, confidence=0.95)
        chi_squared_997, chi2_threshold_997, dof_997, chi_flag_997 = chi(target, df=df, telescopes=telescopes, confidence=0.997)
        chi_squared_68, chi2_threshold_68, dof_68, chi_flag_68 = chi(target, df=df, telescopes=telescopes, confidence=0.68)
        offset_flag = offset_warning(target)
        amplitude_5, lower_percentile_5, upper_percentile_5, mag_amplitude_5 = percentile_amplitude(target, df=df, telescopes=telescopes, tails=5)
        amplitude_1, lower_percentile_1, upper_percentile_1, mag_amplitude_1 = percentile_amplitude(target, df=df, telescopes=telescopes, tails=1)
        largest_amp = largest_amplitude(target, df=df, telescopes=telescopes)
        lombs = compute_lomb_scargle(target, df=df, telescopes=telescopes, auto=False, median=False, subtract_median=False, samples_per_peak=10, report=False)
        best_period = lombs['best_period']
        alarm_level_flag = lombs['alarm_level_flag']
        result = amplitude_per_period(target, best_period, df=df, telescopes=telescopes, report=False)
        std_amp = result['std_amplitude']
        
        results_df.loc[target, 'overall_mean'] = overall_mean
        results_df.loc[target, 'overall_median'] = overall_median
        results_df.loc[target, 'overall_mean_mag'] = overall_mags_mean
        results_df.loc[target, 'overall_median_mag'] = overall_mags_median
        results_df.loc[target, 'offset_flag'] = offset_flag
        results_df.loc[target, 'chi_squared_68'] = chi_squared_68
        results_df.loc[target, 'chi2_threshold_68'] = chi2_threshold_68
        results_df.loc[target, 'chi_flag_68'] = chi_flag_68
        results_df.loc[target, 'chi_squared_95'] = chi_squared_95
        results_df.loc[target, 'chi2_threshold_95'] = chi2_threshold_95
        results_df.loc[target, 'chi_flag_95'] = chi_flag_95
        results_df.loc[target, 'chi_squared_997'] = chi_squared_997
        results_df.loc[target, 'chi2_threshold_997'] = chi2_threshold_997
        results_df.loc[target, 'chi_flag_997'] = chi_flag_997
        results_df.loc[target, 'amplitude_5'] = amplitude_5
        results_df.loc[target, 'lower_percentile_5'] = lower_percentile_5
        results_df.loc[target, 'upper_percentile_5'] = upper_percentile_5
        results_df.loc[target, 'mag_amplitude_5'] = mag_amplitude_5
        results_df.loc[target, 'amplitude_1'] = amplitude_1
        results_df.loc[target, 'lower_percentile_1'] = lower_percentile_1
        results_df.loc[target, 'upper_percentile_1'] = upper_percentile_1
        results_df.loc[target, 'mag_amplitude_1'] = mag_amplitude_1
        results_df.loc[target, 'largest_amp'] = largest_amp
        results_df.loc[target, 'best_period'] = best_period
        results_df.loc[target, 'alarm_level_flag'] = alarm_level_flag
        results_df.loc[target, 'std_amplitude'] = std_amp
        
        # Explicit memory cleanup to prevent accumulation
        del df, telescopes, lombs, result
        del overall_mean, means, overall_median, medians
        del overall_mags_mean, overall_mags_median, mags_means, mags_medians
        del amplitude_5, amplitude_1, largest_amp, std_amp
        
    except Exception as e:
        print(f"Target {target} failed: {e}")
        # Values remain NaN in DataFrame - no action needed
        continue
    
    # More frequent garbage collection for better memory management
    if target % 25 == 0:
        gc.collect()
        # Optional: Memory monitoring
        if target % 100 == 0:
            import psutil, os
            process = psutil.Process(os.getpid())
            memory_mb = process.memory_info().rss / 1024 / 1024
            print(f"Memory usage at target {target}: {memory_mb:.1f} MB")

# Save results
results_df.to_csv('summary_results17.csv', index=False)
print(f"Analysis complete! Results saved to summary_results17.csv")
print(f"Shape: {results_df.shape}")
print(f"Memory usage: {results_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

Target:  0
Offsets for object 0 (g-band: True):
{'k_vs_l': np.float64(nan), 'k_vs_o': np.float64(nan), 'k_vs_p': np.float64(nan), 'k_vs_G': np.float64(nan), 'k_vs_H': np.float64(nan), 'l_vs_k': np.float64(nan), 'l_vs_o': nan, 'l_vs_p': nan, 'l_vs_G': nan, 'l_vs_H': nan, 'o_vs_k': np.float64(nan), 'o_vs_l': nan, 'o_vs_p': nan, 'o_vs_G': nan, 'o_vs_H': nan, 'p_vs_k': np.float64(nan), 'p_vs_l': nan, 'p_vs_o': nan, 'p_vs_G': nan, 'p_vs_H': nan, 'G_vs_k': np.float64(nan), 'G_vs_l': nan, 'G_vs_o': nan, 'G_vs_p': nan, 'G_vs_H': nan, 'H_vs_k': np.float64(nan), 'H_vs_l': nan, 'H_vs_o': nan, 'H_vs_p': nan, 'H_vs_G': nan}
Amplitude metrics:
	Lower 5% percentile: 9.95 mJy, 13.83 mags
	Upper 5% percentile: 10.70 mJy, 13.91 mags
	Percentile amplitude of variability (excluding the brightest and dimmest 5% of data) (g-band: True): 0.75 mJy, 0.08 mags
Amplitude metrics:
	Lower 1% percentile: 9.80 mJy, 13.81 mags
	Upper 1% percentile: 10.87 mJy, 13.92 mags
	Percentile amplitude of variability (excluding

In [ ]:
for target in range(0,150): # 848
# for target in range(0,100):  
    # fit_plotter_flux(target)
    df, teles = offset_corrector(target, additive=False, show=True)
    try:
        lombs = compute_lomb_scargle(target, df=df, telescopes=teles, auto=False, samples_per_peak=10)
        plot_periodogram(target, lombs['frequency'], lombs['power'], lombs['best_frequency'], lombs['false_alarm_level'], lombs['peaks'], lombs['observation_period'], show_best=True)
        plot_phase_fold(target, lombs['best_period'], phase_bins=2, df=df, telescopes=teles)
    except Exception as e:
        print(f"Discarded lightcurve at target {target}: {e}")

### Adding logT, logL to the summary_results files

In [7]:
# stats = pd.read_csv('summary_results02.csv')
# stats = pd.read_csv('summary_results03.csv')
# stats = pd.read_csv('summary_results08.csv')
# stats = pd.read_csv('summary_results07.csv')
# summary_file = 'summary_results12.csv'
summary_file = 'summary_results17.csv'
stats = pd.read_csv(summary_file)

df_lmc = pd.read_csv('/Users/shannonbowes/Research/ysg/candidates_and_lcs/final_lmc_ysgcands.csv', comment='#') # , sep="\\s+"
df_smc = pd.read_csv('/Users/shannonbowes/Research/ysg/candidates_and_lcs/final_smc_ysgcands.csv', comment='#') # , sep="\\s+"

# Create coordinate keys for matching (using a small tolerance for floating point precision)
def round_coords(ra, dec, precision=6):
    """Round coordinates to specified decimal places for robust matching"""
    return (round(ra, precision), round(dec, precision))

# Create coordinate-to-logT/logL mapping from df_smc and df_lmc
coord_to_logT = {}
coord_to_logL = {}

# Add SMC data (indices 0-376 in the combined catalog)
for idx, row in df_smc.iterrows():
    coord_key = round_coords(row['ra'], row['dec'])
    coord_to_logT[coord_key] = row['logT']
    coord_to_logL[coord_key] = row['logL']

# Add LMC data (indices 377+ in the combined catalog)  
for idx, row in df_lmc.iterrows():
    coord_key = round_coords(row['ra'], row['dec'])
    coord_to_logT[coord_key] = row['logT']
    coord_to_logL[coord_key] = row['logL']

# Create new logT and logL columns in summary_df
stats['logT'] = np.nan
stats['logL'] = np.nan

# Match coordinates and assign logT/logL values
matches = 0
for idx, row in stats.iterrows():
    coord_key = round_coords(row['RA'], row['DEC'])
    
    if coord_key in coord_to_logT:
        stats.at[idx, 'logT'] = coord_to_logT[coord_key]
        stats.at[idx, 'logL'] = coord_to_logL[coord_key]
        matches += 1

print(f"Successfully matched {matches} out of {len(stats)} stars")
print(f"Missing logT/logL for {len(stats) - matches} stars")

# Save the updated dataframe back to summary_results09.csv
stats.to_csv(summary_file, index=False)
print(f"Updated {summary_file} with logT and logL columns")

# Show a preview of the new columns
print("\nPreview of updated dataframe:")
print(stats[['RA', 'DEC', 'logT', 'logL']].head(10))

# Check for any missing values
missing_logT = stats['logT'].isna().sum()
missing_logL = stats['logL'].isna().sum()
print(f"\nMissing values - logT: {missing_logT}, logL: {missing_logL}")

Successfully matched 848 out of 848 stars
Missing logT/logL for 0 stars
Updated summary_results17.csv with logT and logL columns

Preview of updated dataframe:
         RA        DEC      logT      logL
0  7.521702 -73.913918  3.773141  4.026600
1  7.746443 -73.741425  3.770847  4.010042
2  7.925977 -73.531670  3.634779  4.044608
3  7.948884 -73.599243  3.629099  4.026778
4  7.980047 -73.578545  3.609289  4.036388
5  7.981742 -73.585571  3.617613  4.323311
6  8.228736 -73.821953  3.794981  4.034392
7  8.560188 -73.619049  3.617889  4.098910
8  8.637665 -73.873337  3.879346  4.206012
9  8.763608 -73.549622  3.626272  4.272656

Missing values - logT: 0, logL: 0


In [15]:
df15 = pd.read_csv('summary_results17.csv')
df16 = pd.read_csv('summary_results16.csv')
counter = 0
variable_diff_counter = 0
total_variable_counter = 0

# comparing summary_results17 to summary_results16
for target in range(0,848):
    # Compare the two DataFrames
    if df15.iloc[target]['best_period'] - df16.iloc[target]['best_period'] > 0.1:
        print(f"DataFrames differ at index {target}")
        print("summary_results17:")
        print(df15.iloc[target]['best_period'])
        print("summary_results16:")
        print(df16.iloc[target]['best_period'])
        counter += 1
        if df15.iloc[target]['alarm_level_flag'] != df16.iloc[target]['alarm_level_flag']:
            variable_diff_counter += 1
        if df15.iloc[target]['alarm_level_flag'] != 1:
            variable_counter += 1

print(f"Total differing entries found: {counter}")
print(f"Total variable differing entries found: {variable_diff_counter}")
print(f"Total variable entries found: {variable_counter}")

DataFrames differ at index 2
summary_results17:
309.2481948431621
summary_results16:
201.54708472066625
DataFrames differ at index 7
summary_results17:
27.929154328293713
summary_results16:
6.9017019001918465
DataFrames differ at index 9
summary_results17:
419.45098250948786
summary_results16:
388.6442267030881
DataFrames differ at index 20
summary_results17:
420.07804524263946
summary_results16:
254.17285333168633
DataFrames differ at index 21
summary_results17:
511.08222110082954
summary_results16:
254.29619338834365
DataFrames differ at index 25
summary_results17:
571.1333794468534
summary_results16:
553.2609933870724
DataFrames differ at index 26
summary_results17:
753.3181307772339
summary_results16:
341.55226943668276
DataFrames differ at index 28
summary_results17:
510.38891932520465
summary_results16:
398.3916114597043
DataFrames differ at index 29
summary_results17:
1102.3752195635443
summary_results16:
955.009064300603
DataFrames differ at index 30
summary_results17:
289.7725

### Old data:

In [ ]:
results = []
for target in range(0,848):
    print("Target: ", target)
    overall_mean, means, overall_median, medians, overall_mags_mean, overall_mags_median, mags_means, mags_medians = mean_med_flux(target, g=True, newfiles=False)
    chi_squared_95, chi2_threshold_95, dof_95, chi_flag_95 = chi(target, g=True, newfiles=False)
    chi_squared_997, chi2_threshold_997, dof_997, chi_flag_997 = chi(target, confidence=0.997, g=True, newfiles=False)
    chi_squared_68, chi2_threshold_68, dof_68, chi_flag_68 = chi(target, confidence=0.68, g=True, newfiles=False)
    # largest_amp = curve_fitter_flux(target)
    offset_flag = offset_warning(target, newfiles=False)
    amplitude_5, lower_percentile_5, upper_percentile_5, mag_amplitude_5 = percentile_amplitude(target, tails=5, g=True, newfiles=False)
    amplitude_1, lower_percentile_1, upper_percentile_1, mag_amplitude_1 = percentile_amplitude(target, tails=1, g=True, newfiles=False)
    # plot_percentile_amplitude(target, tails=1)
    largest_amp = largest_amplitude(target, g=True, newfiles=False)
    # lombs = compute_lomb_scargle(target, auto=True, samples_per_peak=5) #freq_range=(0.0, 0.05),
    # plot_periodogram(lombs['frequency'], lombs['power'], lombs['false_alarm_level'], lombs['peaks'], lombs['observation_period'], auto = True, show_best=True)
    lombs = compute_lomb_scargle(target, auto=False, median=False, subtract_median=False, samples_per_peak=10, report=False, newfiles=False)
    best_period = lombs['best_period']
    alarm_level_flag = lombs['alarm_level_flag']
    result = amplitude_per_period(target, report = False, newfiles=False)
    std_amp = result['std_amplitude']
    # amplitude_per_period_plot(target, result)

    # fit_plotter_flux(target)
    # plot_periodogram(lombs['frequency'], lombs['power'], lombs['false_alarm_level'], lombs['peaks'], lombs['observation_period'], show_best=True)
    # plot_phase_fold(target, phase_bins=2)

        # Get RA and DEC for this target
    RA = coords['ra'][target]
    DEC = coords['dec'][target]

    results.append({
        'RA': RA,
        'DEC': DEC,
        'overall_mean': overall_mean,
        'overall_median': overall_median,
        'overall_mean_mag': overall_mags_mean,
        'overall_median_mag': overall_mags_median,
        'offset_flag': offset_flag,
        'chi_squared_68': chi_squared_68,
        'chi2_threshold_68': chi2_threshold_68,
        'chi_flag_68': chi_flag_68,
        'chi_squared_95': chi_squared_95,
        'chi2_threshold_95': chi2_threshold_95,
        'chi_flag_95': chi_flag_95,
        'chi_squared_997': chi_squared_997,
        'chi2_threshold_997': chi2_threshold_997,
        'chi_flag_997': chi_flag_997,
        'amplitude_5': amplitude_5,
        'lower_percentile_5': lower_percentile_5,
        'upper_percentile_5': upper_percentile_5,
        'mag_amplitude_5': mag_amplitude_5,
        'amplitude_1': amplitude_1,
        'lower_percentile_1': lower_percentile_1,
        'upper_percentile_1': upper_percentile_1,
        'mag_amplitude_1': mag_amplitude_1,
        'largest_amp': largest_amp,
        'best_period': best_period,
        'alarm_level_flag': alarm_level_flag,
        'std_amplitude': std_amp
    })

# print(results)

# Convert to DataFrame and save to CSV
df_results = pd.DataFrame(results)
df_results.to_csv('summary_results07.csv', index=False)

In [ ]:
print(df)

# sorting

In [7]:
# create three new columns in summary_results15.csv called 'cephied_like', 'ir_variable_like', and 'other_variable_like' and leave them empty for now
summary_file = 'summary_results15.csv'
stats = pd.read_csv(summary_file)
stats['cephied_like'] = ''
stats['ir_variable_like'] = ''
stats['other_variable_like'] = ''
stats.to_csv(summary_file, index=False)